In [ ]:
# read in the text 
import pandas as pd 
import csv

df = pd.read_csv('Gutenberg-Poetry.csv', header=0, delimiter=",",  
                       encoding='utf-8')


df.columns =  ['index', 'text', 'gid' ]
print(df.head(100))




In [10]:
lines = []

for row in df.itertuples(index=True, name='Pandas'):
      lines.append(row.text)

In [12]:
print(lines[-100:])

['And straight the spirits that inhabit dream', 'Came round him, and the rain-squall roared its last', 'And bright the wind-vane shifted as it passed.', 'And in the rush of sun and glittering cloud', 'That followed on the storm, he led the way,', 'Fluting the sodden circus through the crowd', 'That trod the city streets in holiday.', 'And lo, a marvellous thing, the gouted clay,', 'Splashed on the waggons and the horses, glowed,', 'They shone like embers as they trod the road.', 'And round the tired horses came the Powers', "That stir men's spirits, waking or asleep,", 'To thoughts like planets and to acts like flowers,', "Out of the inner wisdom's beauty deep:", 'These led the horses, and, as marshalled sheep', 'Fronting a dog, in line, the people stared', 'At those bright waggons led by the bright-haired.', 'And, as they marched, the spirits sang, and all', 'The horses crested to the tune and stept', 'Like centaurs to a passionate festival', 'With shining throats that mantling crinie

In [13]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [14]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [17]:
# clean the text - lowercase, remove stopwords, remove special chars, split into sentences and remove v short sentences
import re

stops = set(stopwords.words('english'))

raw_text = ".".join(lines)
text = raw_text.lower()
text = text.replace("’", "'")
text = re.sub("[^A-Za-z'.]", " ", text)

text_cleaned = " ".join([i for i in text.split(" ") if i not in stops])
text_cleaned = re.sub("[ ]+", " ", text_cleaned)

# removing very short sentences
sentences = text_cleaned.split(".")
sentences = [i.strip() for i in sentences if len(i.strip().split(" ")) > 2]

print(sentences[:10])

# might remove plurals manually just check if without s in the data and replace
# unsure if stopwords were successfully removed, since a lot of top words were borderline could be stopwords, check this



['song hiawatha based legends stories of', 'many north american indian tribes especially the', 'ojibway indians northern michigan wisconsin minnesota', 'they collected henry rowe schoolcraft reknowned', 'schoolcraft married jane bah bahm wawa ge zhe go qua the', 'fur trader shau gus coday way qua woman green', 'prairie daughter waub jeeg white fisher', 'who chief ojibway tribe la pointe wisconsin', 'jane mother credited researched', 'authenticated compiled much material schoolcraft']


In [18]:
# identify phrases that might skew (eg one meaning across two words not two strongly related words)
from gensim.models.phrases import Phrases, Phraser

words_list = [i.split() for i in sentences]

phrases = Phrases(words_list, min_count=30, progress_per=10000)
print(words_list)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [19]:
bigram = Phraser(phrases)
sents = bigram[words_list]

In [20]:
# check most frequent words:
from collections import defaultdict
word_freq = defaultdict(int)
for sent in sents:
    for i in sent:
        word_freq[i] += 1
len(word_freq)
sorted(word_freq, key=word_freq.get, reverse=True)[:10]

['and', 'the', 'to', 'thy', 'but', 'i', 'of', 'that', 'in', 'one']

In [21]:
import multiprocessing
from gensim.models import Word2Vec 


In [22]:
cores = multiprocessing.cpu_count()
cores

2

In [26]:
wtv_model = Word2Vec(min_count=10,
                     window=2,
                     size=300,
                     sample=6e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=20,
                     workers=cores-1)

In [27]:
wtv_model.build_vocab(sents, progress_per=10000)

In [28]:
wtv_model.train(sents, total_examples=wtv_model.corpus_count, epochs=30, report_delay=1)


(199719523, 319534170)

In [29]:
# can save the model etc.
wtv_model.save("word2vec.model")

In [30]:
wtv_model.init_sims(replace=True)

In [31]:
# note, so far have neither stemmed or lemmatized which should do and remake model, and maybe add more data
print(wtv_model.most_similar('lord'))

[('king', 0.7384674549102783), ('god', 0.6446132659912109), ('father', 0.6049135327339172), ('prince', 0.597366452217102), ('son', 0.5629732012748718), ('sire', 0.5251263380050659), ('friend', 0.513664722442627), ('thee', 0.5087589025497437), ('queen', 0.5084806680679321), ('brother', 0.4816444516181946)]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  
